In [1]:
# first run installations and data download
# ! pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file
# ! mkdir -p './data/10k/'
# ! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
# ! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'
# ! mv 'lyft_2021.pdf' 'uber_2021.pdf' './data/10k/'

In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [ ]:
# for use with milvus lite
# ! pip install milvus

In [2]:
from milvus import default_server
default_server.start()

In [15]:
# default_server.stop()
# default_server.cleanup()

In [12]:
from dotenv import load_dotenv
load_dotenv()

import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [16]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/ac"
    )
    ac_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [17]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()
    ac_docs = SimpleDirectoryReader(
        input_files=["./data/10k/AC-Unit.pdf"]
    ).load_data()

    # build index
    vector_store_lyft = MilvusVectorStore(host="localhost", port=default_server.listen_port, dim=1536, collection_name="lyft", overwrite=True)
    vector_store_uber = MilvusVectorStore(host="localhost", port=default_server.listen_port, dim=1536, collection_name="uber", overwrite=True)
    vector_store_ac = MilvusVectorStore(host="localhost", port=default_server.listen_port, dim=1536, collection_name="ac", overwrite=True)
    storage_context_lyft = StorageContext.from_defaults(vector_store=vector_store_lyft)
    storage_context_uber = StorageContext.from_defaults(vector_store=vector_store_uber)
    storage_context_ac = StorageContext.from_defaults(vector_store=vector_store_ac)
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, storage_context=storage_context_lyft)
    uber_index = VectorStoreIndex.from_documents(uber_docs, storage_context=storage_context_uber)
    ac_index = VectorStoreIndex.from_documents(ac_docs, storage_context=storage_context_ac)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
    ac_index.storage_context.persist(persist_dir="./storage/ac")

In [18]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)
ac_engine = ac_index.as_query_engine(similarity_top_k=3)

In [20]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=ac_engine,
        metadata=ToolMetadata(
            name="AC_manual",
            description=(
                "Provides information about AC unit manual instructions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [21]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [22]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [ ]:
response = agent.chat("For the AC unit, how do I change the filter?")
print(str(response))

Thought: I can use the AC_manual tool to find instructions on how to change the filter of an AC unit.
Action: AC_manual
Action Input: {'input': 'How to change the filter of an AC unit?'}
Observation: To change the filter of an AC unit, you should first open the panel and then loosen the clasp of the filter. After that, remove the axile bush of the horizontal louver. Push the filter inward and then raise it to remove it.
Thought: I have found the instructions on how to change the filter of an AC unit.
Answer: To change the filter of an AC unit, follow these steps:
1. Open the panel of the AC unit.
2. Loosen the clasp of the filter.
3. Remove the axile bush of the horizontal louver.
4. Push the filter inward and then raise it to remove it.

Please let me know if there is anything else I can help you with.
To change the filter of an AC unit, follow these steps:
1. Open the panel of the AC unit.
2. Loosen the clasp of the filter.
3. Remove the axile bush of the horizontal louver.
4. Push t

2024-03-02 14:41:43.640 - [BAML_CLIENT] - WARNING: 
Event: ProcessInstructions
-------------------
Prompt
Role: system
Answer the question:
...
Role: assistant
...
Role: user
Convert to this Output JSON Format:
{
  "steps": string[],
  "page": int,
  "warnings": string[]
}

Before outputting the JSON, write a paragraph explaining the most relevant information from the manual to answer the questions.
-------------------
Raw LLM Output (Tokens: prompt=70 output=54)
I'm sorry, but I can't provide the help you're looking for without more information. Could you please provide more details or context? For instance, what manual are you referring to? What questions need to be answered? This information will help me assist you better.
-------------------
Error
DeserializerException: Failed to Deserialize: (1 errors) (0 warnings)
------
Error in Instructions: Failed to parse into Instructions: 3 validation errors for Instructions
steps
  Field required [type=missing, input_value={}, input_type=d

In [24]:
from baml_client import baml as b

result = await b.ProcessInstructions(query="", answer =str(response))

DeserializerException: Failed to Deserialize: (1 errors) (0 warnings)
------
Error in Instructions: Failed to parse into Instructions: 3 validation errors for Instructions
steps
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
page
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
warnings
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
------
Raw:
I'm sorry, but I can't provide the help you're looking for without more information. Could you please provide more details or context? For instance, what manual are you referring to? What questions need to be answered? This information will help me assist you better.